# All features

Bring all features together into a single dataset

In [1]:
# base libraries
import pandas as pd
import numpy as np
import math
import os
import json

In [2]:
# set variable from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
shipping_rot_filename = config['DEFAULT']['shipping_rot_filename']
segment_filename = config['DEFAULT']['segment_filename']
port_loading_filename = config['DEFAULT']['port_loading_filename']
seasonality_filename = config['DEFAULT']['seasonality_filename']
delay_filename = config['DEFAULT']['delay_filename']
weather_data_filename = config['DEFAULT']['weather_data_filename']
ship_dynamics_filename = config['DEFAULT']['ship_dynamics_filename']
features_filename = config['DEFAULT']['features_filename']

In [3]:
# import data
dtype_dic = {'MMSI':int,'dt':'str', 'lat':'float', 'long':'float','SOG':'float', 'rot':'float', 
             'Type':'str', 'gross_tonnage':'float','vessel_name':'str', 'ETA':'str', 'POC_LOCODE':'str',
             'last_port_LOCODE':'str', 'next_port_LOCODE':'str', 'status':'str','voyage_id':'float','tripid':int,
            'in_hazmat':'str','out_hazmat':'str'}
parse_dates = ['dt', 'ETA']

shipping_data = pd.read_csv(processing_path + shipping_rot_filename,header = 0,delimiter = ',',dtype = dtype_dic, parse_dates=parse_dates)

## Merge delays data

In [4]:
# import delays data
delays = pd.read_csv(processing_path + delay_filename, header = 0,delimiter = ',')
delays['dt'] = pd.to_datetime(delays['dt'])

In [5]:
shipping_data = shipping_data.merge(delays[['MMSI','dt','arrivalDelay','arrivalDelayMin','delay15','delay30',
                                            'delay60','delay90','delay120','previous_delays','ETA_new']], how = 'inner', 
                                    on = ['MMSI','dt'])

## Ship Type

In [6]:
shipping_data['Type'] = shipping_data['Type'].map({'tug':'tug', 'container ship':'container', 
                                                   'pilot':'pilot', 'ro-ro cargo ship':'cargo_ship', 
                                                   'hopper dredger':'dredger', 
                                                   'general cargo ship':'general_cargo', 
                                                   'passenger/ro-ro cargo ship':'passenger', 
                                                   'work/repair vessel':'work_vessel', 'sar':'sar', 
                                                   'unknown type':'unknown'})

# create one hot encoding for each ship type
one_hot = pd.get_dummies(shipping_data['Type'],prefix = 'type')

In [7]:
# merge dummy fields back on
shipping_data = shipping_data.merge(one_hot, left_index = True,right_index = True, how = 'inner') 

## Segments

In [8]:
segments = pd.read_csv(processing_path + segment_filename, header = 0,delimiter = ',')
segments['dt'] = pd.to_datetime(segments['dt'])

In [9]:
segments['seg_desc'] = segments['segment'].map({0:'Trans_general', 1:'Docking_terminal',2:'Docking_mid', 3:'Docked',
                                               4:'Trans_border', 5:'Docking_inital'})

# create one hot encoding for each segment
one_hot_seg = pd.get_dummies(segments['seg_desc'],prefix = 'seg')

In [10]:
# merge dummy fields back on
shipping_data = shipping_data.merge(one_hot_seg, left_index = True,right_index = True, how = 'inner') 

## Seasonality

In [11]:
seasonality = pd.read_csv(processing_path + seasonality_filename, header = 0,delimiter = ',')
seasonality['dt'] = pd.to_datetime(seasonality['dt'])

In [12]:
shipping_data = shipping_data.merge(seasonality, how = 'inner', on = ['MMSI','dt'])

## Merge port loading data

In [13]:
port_loading = pd.read_csv(processing_path + port_loading_filename, header = 0,delimiter = ',')
port_loading['start_window'] = pd.to_datetime(port_loading['start_window'])
port_loading['end_window'] = pd.to_datetime(port_loading['end_window'])

In [14]:
shipping_data['start_window'] = shipping_data['dt'].dt.floor('1min')
shipping_data['end_window'] = shipping_data['dt'].dt.ceil('1min')

In [15]:
shipping_data = shipping_data.merge(port_loading, on = ['start_window','end_window'], how = 'inner')

## Merge weather data

In [16]:
weather = pd.read_csv(processing_path + weather_data_filename, header = 0,delimiter = ',')
weather['YEARMODA'] = pd.to_datetime(weather['YEARMODA'])
weather['YEARMODA'] = weather['YEARMODA'].dt.date

In [17]:
# give some more meaningful names
rename_dict = {'TEMP':'temp',
               'DEWP':'dew_point',
               'SLP':'sea_lev_pressure',
               'STP':'station_pressure',
               'VISIB':'visibility',
               'WDSP':'wind_speed',
               'MXSPD':'max_wind_speed',
               'MAX':'max_temp',
               'MIN':'min_temp',
               'FRSHTT':'fog_rain_driz_snow_ice'}
weather.rename(columns = rename_dict, inplace = True)
weather.drop(['STN'],axis = 1,inplace = True)

In [18]:
# add the merge key to the features dataset
shipping_data['YEARMODA'] = shipping_data['dt'].dt.date

In [19]:
shipping_data = shipping_data.merge(weather, on = 'YEARMODA', how = 'inner')

## Ship dynamics

In [20]:
ship_dynamics = pd.read_csv(processing_path + ship_dynamics_filename, header = 0,delimiter = ',')
ship_dynamics['dt'] = pd.to_datetime(ship_dynamics['dt'])

In [21]:
shipping_data = shipping_data.merge(ship_dynamics[['MMSI','dt','accel','port_dist']], how = 'left', on = ['MMSI','dt'])

## Export Data

In [22]:
shipping_data.to_csv(processing_path + features_filename,header=True,index=False,sep=',')

In [23]:
shipping_data.describe()

,MMSI,SOG,gross_tonnage,lat,long,voyage_id,tripid,rot,arrivalDelayMin,previous_delays,...,sea_lev_pressure,station_pressure,visibility,wind_speed,max_wind_speed,max_temp,min_temp,fog_rain_driz_snow_ice,accel,port_dist
count,1.131759e+06,1.131759e+06,1.131759e+06,1.131759e+06,1.131759e+06,1.131759e+06,1.131759e+06,1.128591e+06,1.131759e+06,561732.000000,...,1.125808e+06,1.125808e+06,1.125808e+06,1.125808e+06,1.125808e+06,1.125808e+06,1.125808e+06,1.125808e+06,1.130161e+06,1.003960e+06
mean,3.763103e+08,1.390241e+00,4.858576e+04,5.195493e+01,1.301660e+00,1.088642e+06,2.717542e+01,5.388764e+00,9.855130e+01,2.762917,...,1.019017e+03,1.018771e+03,3.450480e+01,8.663079e+00,1.350797e+01,1.462448e+01,7.401772e+00,1.597755e+04,1.286545e-04,1.771862e+03
std,1.666183e+08,3.461391e+00,5.840190e+04,6.938276e-03,8.656976e-03,3.607977e+04,6.473060e+01,7.714307e+00,1.028975e+03,4.384490,...,9.430611e+00,9.434266e+00,1.380212e+02,2.910903e+00,3.776399e+00,6.527290e+00,5.751391e+00,3.120908e+04,1.166457e-01,2.673516e+03
min,2.093220e+08,0.000000e+00,0.000000e+00,5.193531e+01,1.281933e+00,1.001116e+06,1.000000e+00,0.000000e+00,-1.236000e+04,0.000000,...,9.882000e+02,9.878000e+02,3.000000e-01,1.800000e+00,5.100000e+00,1.000000e+00,-5.000000e+00,0.000000e+00,-1.460000e+01,0.000000e+00
25%,2.299280e+08,0.000000e+00,6.326000e+03,5.194953e+01,1.294683e+00,1.064553e+06,4.000000e+00,2.330485e-01,-5.700000e+01,0.000000,...,1.014000e+03,1.013700e+03,9.400000e+00,6.500000e+00,1.110000e+01,1.000000e+01,3.000000e+00,0.000000e+00,0.000000e+00,2.506210e+02
50%,3.518190e+08,0.000000e+00,2.419600e+04,5.195705e+01,1.302205e+00,1.090350e+06,8.000000e+00,1.930199e+00,1.100000e+01,1.000000,...,1.019300e+03,1.019000e+03,1.640000e+01,8.200000e+00,1.300000e+01,1.400000e+01,7.000000e+00,1.000000e+04,0.000000e+00,6.059190e+02
75%,5.631080e+08,1.400000e+00,7.288400e+04,5.196029e+01,1.309360e+00,1.116384e+06,2.000000e+01,7.910323e+00,1.260000e+02,3.000000,...,1.025300e+03,1.025000e+03,2.090000e+01,1.020000e+01,1.590000e+01,2.100000e+01,1.200000e+01,1.000000e+04,0.000000e+00,2.187710e+03
max,6.360926e+08,1.023000e+02,2.100000e+05,5.196582e+01,1.320850e+00,1.163320e+06,4.310000e+02,3.469213e+02,4.302500e+04,24.000000,...,1.043300e+03,1.043000e+03,9.999000e+02,2.030000e+01,3.090000e+01,3.000000e+01,1.900000e+01,1.100000e+05,1.278750e+01,1.336753e+04
